# Borrower reliability research

To study whether the marital status and the number of children of the client affects the fact of repayment of the loan on time. The results of the analysis will be further used by the bank's credit department to build a credit scoring model.

### Description of attributes:

- children - number of children in the family;
- days_employed - total length of service in days;
- dob_years - client's age in years;
- education - education level of the client;
- education_id - education level identifier;
- family_status - family status;
- family_status_id - family status identifier;
- gender - client's gender;
- income_type - type of employment;
- debt - whether the client had debts on loan repayment;
- total_income - monthly income;
- purpose - purpose of obtaining a loan.

### Libraries and modules

In [1]:
import pandas as pd
from pymystem3 import Mystem
from collections import Counter

### Contents

1. Study of general information about the data
2. Data preprocessing:
    - Skip processing
    - Data type replacement
    - Duplicate processing
    - Lemmatization
    - Data categorization
    - Additional data exploration
3. Analytics:
    - Is there a relationship between having children and loan repayment on time?
    - Is there a correlation between marital status and loan repayment on time?
    - Is there a relationship between income level and loan repayment on time?
    - How do different loan purposes affect loan repayment on time?
4. General conclusion

## Study of general information about the data

In [2]:
data = pd.read_csv('data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


#### Conclusion

In the general information of this table we can see that there are columns with omissions (days_employed, total_income), as 19351 values in them are filled in while the other columns are 21525. It is also interesting that the days_employed column contains negative values.

## Data preprocessing

### Processing of passed values

Gaps may occur due to technical reasons or human error.

Gaps are divided into three types:
- completely random - skips whose probability of occurrence does not depend on other values in the data set
- random - skips, the probability of occurrence of which depends on other values in the data set
- non-random - skips, the probability of occurrence of which depends on other values and on the value of its own column.

In [4]:
data.isnull().sum()
# the combination of isnull and sum methods gives information about how many skips are in each column

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

The first question to answer is, is there a pattern to the appearance of omissions in the days_employed and total_income columns?

- you can see that gaps are made in quantitative variables, since the data type of these columns is float64.
- I hypothesize that there is a pattern between the omissions in these columns

Hypothesis: if a person does not indicate how many days he/she worked, it means that he/she did not work, so he/she does not fill in the income field. Based on this reasoning, the omissions in the days_employed column are completely random and the omissions in the total_income column are random.

In [5]:
data.loc[data['days_employed'].isnull() == True].head(10)
# output rows in the table that contain gaps in the days_employed column

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


Analyzing the data, there is a direct correlation between days_employed and total_income. However, the assumption that the person did not work a day, so he/she did not indicate income is not correct, because if we look at the sources of income of people in this sample, there will be: employee, pensioner, civil servant and others. In this case, the person could not not work and not receive income. Then we can reason from the idea that people hide their income and do not want to say it, as well as the number of days worked, or one of the employees working in the bank, collecting this information about clients, forgot to put it in the table, or simply did not ask the client about it. A technical reason cannot be ruled out.

I will fill in the gaps in the total_income column with the median value for each of the categories of income sources, pre-grouped by income_type, since the same source of income can have a strong variation in values, and the gaps in the days_employed column with the median value as well.

However, we remember that the days_employed column contains negative values, which cannot be according to the meaning of the variable. Perhaps this error was made due to technical problems that added a minus to the fields of this column during uploading. I will change the negative values to positive ones.

In [6]:
def abs_days_employed(var): # a function that outputs the modulus values of the passed number
    return abs(var)

data['days_employed'] = data.loc[data['days_employed'].isnull() == False, 'days_employed'].apply(abs_days_employed)
# apply the abs_days_employed function using the apply method to the days_employed column

In [7]:
dict_income_type = data.groupby('income_type').agg({'total_income': ['median'], 'days_employed': ['median'] })
# group the table by income_type column using the groupby method and aggregate using the agg method
dict_income_type

,total_income,days_employed
,median,median
income_type,,
безработный,131339.751676,366413.652744
в декрете,53829.130729,3296.759962
госслужащий,150447.935283,2689.368353
компаньон,172357.950966,1547.382223
пенсионер,118514.486412,365213.306266
предприниматель,499163.144947,520.848083
сотрудник,142594.396847,1574.202821
студент,98201.625314,578.751554


In [8]:
data = data.fillna(value='') # replace the blanks with ''
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.42261,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.07787,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.97192,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [9]:
data_len = data.shape[0]

for x in range(data_len): # replace empty values with median monthly income values
    try:
        if data.loc[x, 'total_income'] == '':
            x_income_type = data.loc[x, 'income_type']
            median_income = dict_income_type.loc[x_income_type, 'total_income'].to_list()[0]
            data.loc[x, 'total_income'] = median_income
    except:
        print('Error')

I use the try-except construct to minimize the occurrence of errors. This algorithm checks for each row to see if there is a missing value in the 'total_income' column. If there is a missing value, we store the type of income source in the variable x_income_type. After that I turn to the dictionary source employment type-median value of monthly income for this type of employment and select median monthly income (median_income_type), which I insert into the value of this row in the column 'total_income'.

In [10]:
data.loc[data['total_income'] == ''] # check for empty values in the total_income column

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


In [11]:
data.loc[data['days_employed'] == '', ['days_employed', 'total_income']].head(10)

,days_employed,total_income
12,,118514.486412
26,,150447.935283
29,,118514.486412
41,,150447.935283
55,,118514.486412
65,,172357.950966
67,,118514.486412
72,,150447.935283
82,,142594.396847
83,,142594.396847


In [12]:
data_len = data.shape[0]

for x in range(data_len): # replace skips with the median value of years of service
    try:
        if data.loc[x, 'days_employed'] == '':
            x_income_type = data.loc[x, 'income_type']
            median_days_employed = dict_income_type.loc[x_income_type, 'days_employed'].to_list()[0]
            data.loc[x, 'days_employed'] = median_days_employed
    except:
        print('Error')

In [13]:
data.loc[data['days_employed'] == '', ['days_employed', 'total_income']].head(10)
# I check if there are empty values in the days_employed column

,days_employed,total_income


#### Conclusion

In total, I replaced omissions in the total_income and days_employed columns, filling them with median values, grouped by source of income (income_type). Determined that the pattern of skips in these columns is regularized. The data in the days_employed column were also corrected from negative to positive values.

### Data type replacement

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  object
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  object
 11  purpose           21525 non-null  object
dtypes: int64(5), object(7)
memory usage: 2.0+ MB


You can see that the days_employed and total_income columns should be converted from string type to float64 type, because they contain data of numeric type.

In [15]:
try:
    data['days_employed'] = pd.to_numeric(data['days_employed']) 
    data['total_income'] = pd.to_numeric(data['total_income'])
except:
    print('Error')

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


#### Conclusion

Changed data types from lowercase to float64 of columns: days_employed and total_income.

### Duplicate processing

Look for duplicates in the columns: education, family_status, gender, income_type. The other column types have a quantitative data type, the purpose column contains a textual reason.

In [17]:
# I'm looking at the education column
print(data['education'].duplicated().sum())
print(data['education'].value_counts())

21510
среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64


The column contains values of the same word written with different case characters. Let's convert all the characters of the values in this column to lower case.

In [18]:
data['education'] = data['education'].str.lower()
# use the str.lower method to lowercase all values in the column
data['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [19]:
data['family_status'] = data['family_status'].str.lower()
data['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

Essentially common-law marriage and unmarried/unmarried are the same thing, so let's just leave unmarried/unmarried.

In [20]:
data.loc[data['family_status'] == 'гражданский брак', 'family_status'] = 'женат / замужем'
data['family_status'].value_counts()

женат / замужем          16557
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [21]:
data['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Interestingly, there is one line that lists gender as XNA. What is that? Let's replace it with a mode value.

In [22]:
data.loc[data['gender'] == 'XNA', 'gender'] = data['gender'].mode()
data['gender'] = data['gender'].str.lower()
data.loc[data['gender'] == 'XNA', 'gender']

Series([], Name: gender, dtype: object)

In [23]:
data['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

In [24]:
data.duplicated().sum()

71

In [25]:
data = data.drop_duplicates().reset_index(drop=True)
data.duplicated().sum()

0

#### Conclusion

Unnecessary values in the education column were removed. One value in the gender column was removed. Since the values in the columns are categorical, there is no need to look for duplicates in the general sense.

### Lemmatization

Lemmatization is the process of reducing a word to its dictionary form.

In [26]:
from pymystem3 import Mystem
from collections import Counter
m = Mystem()
words = []
rows = data.shape[0]
try: # create a list of unique words from all reasons for taking credit
    for row in range(rows):
        list_purpose = data.loc[row, 'purpose']
        words += m.lemmatize(data.loc[row, 'purpose'])
except:
    print('Error')

words = Counter(words)
print(words.keys())

dict_keys(['покупка', ' ', 'жилье', '\n', 'приобретение', 'автомобиль', 'дополнительный', 'образование', 'сыграть', 'свадьба', 'операция', 'с', 'на', 'проведение', 'для', 'семья', 'недвижимость', 'коммерческий', 'жилой', 'строительство', 'собственный', 'подержать', 'свой', 'со', 'заниматься', 'сделка', 'подержанный', 'получение', 'высокий', 'профильный', 'сдача', 'ремонт'])


In [27]:
list_target_words = ['жилье', 'автомобиль', 'образование', 'свадьба', 'недвижимость', 'строительство', 'ремонт']
# of the unique ones obtained, I create a list of important and keywords that characterize the cause

In [28]:
rows = data.shape[0]

for row in range(rows):
    try: # create a column that will store the keyword that characterizes the cause
        lemmas = m.lemmatize(data.loc[row, 'purpose'])
        for lemma in lemmas:
            if lemma in list_target_words:
                data.loc[row, 'purpose_unique'] = lemma
    except:
        print('Error')

In [29]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_unique
0,1,8437.673028,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875.639453,покупка жилья,жилье
1,1,4024.803754,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080.014102,приобретение автомобиля,автомобиль
2,0,5623.422610,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885.952297,покупка жилья,жилье
3,3,4124.747207,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628.550329,дополнительное образование,образование
4,0,340266.072047,53,среднее,1,женат / замужем,1,f,пенсионер,0,158616.077870,сыграть свадьбу,свадьба
5,0,926.185831,27,высшее,0,женат / замужем,1,m,компаньон,0,255763.565419,покупка жилья,жилье
6,0,2879.202052,43,высшее,0,женат / замужем,0,f,компаньон,0,240525.971920,операции с жильем,жилье
7,0,152.779569,50,среднее,1,женат / замужем,0,m,сотрудник,0,135823.934197,образование,образование
8,2,6929.865299,35,высшее,0,женат / замужем,1,f,сотрудник,0,95856.832424,на проведение свадьбы,свадьба
9,0,2188.756445,41,среднее,1,женат / замужем,0,m,сотрудник,0,144425.938277,покупка жилья для семьи,жилье


In [30]:
data.loc[data['purpose_unique'] == 'жилье', 'purpose_unique'] = 'недвижимость'
print(data['purpose_unique'].value_counts())
print(f"The sum of the values is equal to: {data['purpose_unique'].value_counts().sum()}")
print(f'Total rows in the table: {data.shape[0]}')

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_unique, dtype: int64
The sum of the values is equal to: 21454
Total rows in the table: 21454


#### Conclusion

We have a column with a single word for the reason for taking a loan from a customer's proposal.

### Data categorization

Data categorization is divided into two types: categorization by type and categorization by data merging.

In [31]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21454 non-null  int64  
 1   days_employed     21454 non-null  float64
 2   dob_years         21454 non-null  int64  
 3   education         21454 non-null  object 
 4   education_id      21454 non-null  int64  
 5   family_status     21454 non-null  object 
 6   family_status_id  21454 non-null  int64  
 7   gender            21453 non-null  object 
 8   income_type       21454 non-null  object 
 9   debt              21454 non-null  int64  
 10  total_income      21454 non-null  float64
 11  purpose           21454 non-null  object 
 12  purpose_unique    21454 non-null  object 
dtypes: float64(2), int64(5), object(6)
memory usage: 2.1+ MB


The following table columns fall under categorization by type: family_status, education, income_type, purpose_unique.
The following columns are categorized by data aggregation: children, dob_years.

In the purpose_unique column, let's name one word: real estate and housing.

In [32]:
data.loc[data['purpose_unique'] == 'жилье', 'purpose_unique'] = 'недвижимость'
data['purpose_unique'].value_counts()

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_unique, dtype: int64

Let's make the following categorization of data - according to the number of children in the family we can divide it into childless (0 children), small (1-2 children) and large (3+ children).

Do unmarried/unmarried people have children?

In [33]:
data.loc[data['family_status'] == 'не женат / не замужем', 'children'].value_counts()

 0     2262
 1      449
 2       75
 20       9
 3        8
-1        5
 4        2
Name: children, dtype: int64

Yes, there are children, and some even have 20 children! Some have -1 child :) It's worth looking at all people, not a sample of unmarried/unmarried people.

In [34]:
data['children'].value_counts()

 0     14091
 1      4808
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

The 20 children can be explained very simply: a 0 was added to the number 2 during data entry or data unloading. Where the -1 child was assigned a '-' when uploading-loading the data. Both of these errors could have been the result of a technical problem or human error. Let's fix it.

In [35]:
data.loc[data['children'] == 20, 'children'] = 2
data.loc[data['children'] == -1, 'children'] = 1
data['children'].value_counts()

0    14091
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

In [36]:
# I consider a complete family: there is one or more child and marital status: married/unmarried, common-law, single/unmarried.
# Incomplete family: there is one or more child and marital status: divorced, widow/widower. No family: no child and marital status: any.
# Childless family: 0 children, small family: 1-2 children, large family: 3+ children.
def family_type_by_children_parents(row):
    number_of_childrens = row['children']
    family_status = row['family_status']
    if number_of_childrens == 0:
        return 'Не семья'
    if 1 <= number_of_childrens <= 2:
        if family_status in ['женат / замужем', 'не женат / не замужем']:
            return 'полная малодетная семья'
        return 'неполная малодетная семья'
    if number_of_childrens > 2:
        if family_status in ['женат / замужем', 'не женат / не замужем']:
            return 'полная многодетная семья'
        return 'неполная многодетная семья'
    
data['family_type_by_children_parents'] = data.apply(family_type_by_children_parents, axis=1)
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_unique,family_type_by_children_parents
0,1,8437.673028,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875.639453,покупка жилья,недвижимость,полная малодетная семья
1,1,4024.803754,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080.014102,приобретение автомобиля,автомобиль,полная малодетная семья
2,0,5623.422610,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885.952297,покупка жилья,недвижимость,Не семья
3,3,4124.747207,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628.550329,дополнительное образование,образование,полная многодетная семья
4,0,340266.072047,53,среднее,1,женат / замужем,1,f,пенсионер,0,158616.077870,сыграть свадьбу,свадьба,Не семья
5,0,926.185831,27,высшее,0,женат / замужем,1,m,компаньон,0,255763.565419,покупка жилья,недвижимость,Не семья
6,0,2879.202052,43,высшее,0,женат / замужем,0,f,компаньон,0,240525.971920,операции с жильем,недвижимость,Не семья
7,0,152.779569,50,среднее,1,женат / замужем,0,m,сотрудник,0,135823.934197,образование,образование,Не семья
8,2,6929.865299,35,высшее,0,женат / замужем,1,f,сотрудник,0,95856.832424,на проведение свадьбы,свадьба,полная малодетная семья
9,0,2188.756445,41,среднее,1,женат / замужем,0,m,сотрудник,0,144425.938277,покупка жилья для семьи,недвижимость,Не семья


In [37]:
data['family_type_by_children_parents'].value_counts()

Не семья                      14091
полная малодетная семья        6479
неполная малодетная семья       504
полная многодетная семья        361
неполная многодетная семья       19
Name: family_type_by_children_parents, dtype: int64

Let's get rid of data redundancy and simplify data handling. Let's start with family_status.

In [38]:
dict_family_status = data[['family_status', 'family_status_id']] # a dictionary that stores the family status and its index
dict_family_status.head(10)

,family_status,family_status_id
0,женат / замужем,0
1,женат / замужем,0
2,женат / замужем,0
3,женат / замужем,0
4,женат / замужем,1
5,женат / замужем,1
6,женат / замужем,0
7,женат / замужем,0
8,женат / замужем,1
9,женат / замужем,0


In [39]:
dict_family_status = dict_family_status.drop_duplicates().reset_index(drop=True)
dict_family_status.head(10)

,family_status,family_status_id
0,женат / замужем,0
1,женат / замужем,1
2,вдовец / вдова,2
3,в разводе,3
4,не женат / не замужем,4


Strange that married/married was repeated twice. This error occurred because the civil marriage was replaced and its id was not.

In [40]:
data.loc[data['family_status_id'] == 1, 'family_status_id'] = 0
dict_family_status = data[['family_status', 'family_status_id']]
dict_family_status = dict_family_status.drop_duplicates().reset_index(drop=True)
dict_family_status.head(10)

,family_status,family_status_id
0,женат / замужем,0
1,вдовец / вдова,2
2,в разводе,3
3,не женат / не замужем,4


In [41]:
data = data.drop(['family_status'], axis='columns')
data.head(5)

,children,days_employed,dob_years,education,education_id,family_status_id,gender,income_type,debt,total_income,purpose,purpose_unique,family_type_by_children_parents
0,1,8437.673028,42,высшее,0,0,f,сотрудник,0,253875.639453,покупка жилья,недвижимость,полная малодетная семья
1,1,4024.803754,36,среднее,1,0,f,сотрудник,0,112080.014102,приобретение автомобиля,автомобиль,полная малодетная семья
2,0,5623.422610,33,среднее,1,0,m,сотрудник,0,145885.952297,покупка жилья,недвижимость,Не семья
3,3,4124.747207,32,среднее,1,0,m,сотрудник,0,267628.550329,дополнительное образование,образование,полная многодетная семья
4,0,340266.072047,53,среднее,1,0,f,пенсионер,0,158616.077870,сыграть свадьбу,свадьба,Не семья


Likewise with education.

In [42]:
try:
    dict_education = data[['education', 'education_id']]
    dict_education = dict_education.drop_duplicates().reset_index(drop=True)
    data = data.drop(['education'], axis='columns')
except:
    print('That operations already have been done.')

In [43]:
dict_education

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


In [44]:
data.head(5)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,purpose_unique,family_type_by_children_parents
0,1,8437.673028,42,0,0,f,сотрудник,0,253875.639453,покупка жилья,недвижимость,полная малодетная семья
1,1,4024.803754,36,1,0,f,сотрудник,0,112080.014102,приобретение автомобиля,автомобиль,полная малодетная семья
2,0,5623.422610,33,1,0,m,сотрудник,0,145885.952297,покупка жилья,недвижимость,Не семья
3,3,4124.747207,32,1,0,m,сотрудник,0,267628.550329,дополнительное образование,образование,полная многодетная семья
4,0,340266.072047,53,1,0,f,пенсионер,0,158616.077870,сыграть свадьбу,свадьба,Не семья


I will do the same with the income_type, purpose_unique and family_type_by_children_parents columns, but before that I will add the id columns to the data table.

In [45]:
list_income_type = data['income_type'].unique()
for i in range(len(list_income_type)):
    data.loc[data['income_type'] == list_income_type[i], 'income_type_id'] = i
data.head(5)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,purpose_unique,family_type_by_children_parents,income_type_id
0,1,8437.673028,42,0,0,f,сотрудник,0,253875.639453,покупка жилья,недвижимость,полная малодетная семья,0.0
1,1,4024.803754,36,1,0,f,сотрудник,0,112080.014102,приобретение автомобиля,автомобиль,полная малодетная семья,0.0
2,0,5623.422610,33,1,0,m,сотрудник,0,145885.952297,покупка жилья,недвижимость,Не семья,0.0
3,3,4124.747207,32,1,0,m,сотрудник,0,267628.550329,дополнительное образование,образование,полная многодетная семья,0.0
4,0,340266.072047,53,1,0,f,пенсионер,0,158616.077870,сыграть свадьбу,свадьба,Не семья,1.0


In [46]:
data['income_type_id'] = data['income_type_id'].astype('int') # changing the data type from float to integer
dict_income_type = data[['income_type', 'income_type_id']]
dict_income_type = dict_income_type.drop_duplicates().reset_index(drop=True)
data = data.drop(['income_type'], axis='columns')
dict_income_type

,income_type,income_type_id
0,сотрудник,0
1,пенсионер,1
2,компаньон,2
3,госслужащий,3
4,безработный,4
5,предприниматель,5
6,студент,6
7,в декрете,7


In [47]:
data.head(5)

,children,days_employed,dob_years,education_id,family_status_id,gender,debt,total_income,purpose,purpose_unique,family_type_by_children_parents,income_type_id
0,1,8437.673028,42,0,0,f,0,253875.639453,покупка жилья,недвижимость,полная малодетная семья,0
1,1,4024.803754,36,1,0,f,0,112080.014102,приобретение автомобиля,автомобиль,полная малодетная семья,0
2,0,5623.422610,33,1,0,m,0,145885.952297,покупка жилья,недвижимость,Не семья,0
3,3,4124.747207,32,1,0,m,0,267628.550329,дополнительное образование,образование,полная многодетная семья,0
4,0,340266.072047,53,1,0,f,0,158616.077870,сыграть свадьбу,свадьба,Не семья,1


purpose_unique

I'll write functions to make it easier to work with other columns.

In [48]:
def id_column_for_categorical_column(name, df):
    list_column_name = df[name].unique()
    for i in range(len(list_column_name)):
        df.loc[df[name] == list_column_name[i], f'{name}_id'] = i
    df[f'{name}_id'] = df[f'{name}_id'].astype('int')
    return df
    
data = id_column_for_categorical_column('purpose_unique', data)
data.head(5)

,children,days_employed,dob_years,education_id,family_status_id,gender,debt,total_income,purpose,purpose_unique,family_type_by_children_parents,income_type_id,purpose_unique_id
0,1,8437.673028,42,0,0,f,0,253875.639453,покупка жилья,недвижимость,полная малодетная семья,0,0
1,1,4024.803754,36,1,0,f,0,112080.014102,приобретение автомобиля,автомобиль,полная малодетная семья,0,1
2,0,5623.422610,33,1,0,m,0,145885.952297,покупка жилья,недвижимость,Не семья,0,0
3,3,4124.747207,32,1,0,m,0,267628.550329,дополнительное образование,образование,полная многодетная семья,0,2
4,0,340266.072047,53,1,0,f,0,158616.077870,сыграть свадьбу,свадьба,Не семья,1,3


In [49]:
def create_dict_for_categorical_column(id_column, column, df):
    dict_column = df[[column, id_column]]
    dict_column = dict_column.drop_duplicates().reset_index(drop=True)
    return dict_column

dict_purpose_unique = create_dict_for_categorical_column('purpose_unique', 'purpose_unique_id', data)
dict_purpose_unique

,purpose_unique_id,purpose_unique
0,0,недвижимость
1,1,автомобиль
2,2,образование
3,3,свадьба


In [50]:
data = data.drop(['purpose', 'purpose_unique'], axis='columns')
data.head(5)

,children,days_employed,dob_years,education_id,family_status_id,gender,debt,total_income,family_type_by_children_parents,income_type_id,purpose_unique_id
0,1,8437.673028,42,0,0,f,0,253875.639453,полная малодетная семья,0,0
1,1,4024.803754,36,1,0,f,0,112080.014102,полная малодетная семья,0,1
2,0,5623.422610,33,1,0,m,0,145885.952297,Не семья,0,0
3,3,4124.747207,32,1,0,m,0,267628.550329,полная многодетная семья,0,2
4,0,340266.072047,53,1,0,f,0,158616.077870,Не семья,1,3


family_type_by_children_parents

In [51]:
data = id_column_for_categorical_column('family_type_by_children_parents', data)
data.head(5)

,children,days_employed,dob_years,education_id,family_status_id,gender,debt,total_income,family_type_by_children_parents,income_type_id,purpose_unique_id,family_type_by_children_parents_id
0,1,8437.673028,42,0,0,f,0,253875.639453,полная малодетная семья,0,0,0
1,1,4024.803754,36,1,0,f,0,112080.014102,полная малодетная семья,0,1,0
2,0,5623.422610,33,1,0,m,0,145885.952297,Не семья,0,0,1
3,3,4124.747207,32,1,0,m,0,267628.550329,полная многодетная семья,0,2,2
4,0,340266.072047,53,1,0,f,0,158616.077870,Не семья,1,3,1


In [52]:
dict_family_type_by_children_parents = create_dict_for_categorical_column('family_type_by_children_parents', 'family_type_by_children_parents_id', data)
dict_family_type_by_children_parents

,family_type_by_children_parents_id,family_type_by_children_parents
0,0,полная малодетная семья
1,1,Не семья
2,2,полная многодетная семья
3,3,неполная малодетная семья
4,4,неполная многодетная семья


In [53]:
data = data.drop(['family_type_by_children_parents'], axis='columns')
data.head(5)

,children,days_employed,dob_years,education_id,family_status_id,gender,debt,total_income,income_type_id,purpose_unique_id,family_type_by_children_parents_id
0,1,8437.673028,42,0,0,f,0,253875.639453,0,0,0
1,1,4024.803754,36,1,0,f,0,112080.014102,0,1,0
2,0,5623.422610,33,1,0,m,0,145885.952297,0,0,1
3,3,4124.747207,32,1,0,m,0,267628.550329,0,2,2
4,0,340266.072047,53,1,0,f,0,158616.077870,1,3,1


#### Conclusion

Families were categorized by the number of children and parents and categorized by the reason for taking a loan. Also categories by type were put into separate tables: education, family_status, income_type, purpose_unique, family_type_by_children_parents.

### Additional data exploration

It remains to get to know the data in the days_employed and dob_years columns. I will start with days_employed.

Obviously, a person cannot have a total work record in days at least more than 30000 days exactly (that's 83+ years of working every day). Let's reduce the boundary value to 10000 days (27+ years of work every day).

In [54]:
data.loc[data['days_employed'] >= 10000].shape

(4091, 11)

Let's look at values that have >= 100000.

In [55]:
data.loc[data['days_employed'] >= 100000].shape

(3831, 11)

There is clearly a data upload-loading error here. Because there can't be such a length of service. To correct this error, let's divide each number in this sample by 100.

In [56]:
data.loc[data['days_employed'] >= 100000, 'days_employed'] = data.loc[data['days_employed'] >= 100000, 'days_employed'] / 100
data.head(5)

,children,days_employed,dob_years,education_id,family_status_id,gender,debt,total_income,income_type_id,purpose_unique_id,family_type_by_children_parents_id
0,1,8437.673028,42,0,0,f,0,253875.639453,0,0,0
1,1,4024.803754,36,1,0,f,0,112080.014102,0,1,0
2,0,5623.422610,33,1,0,m,0,145885.952297,0,0,1
3,3,4124.747207,32,1,0,m,0,267628.550329,0,2,2
4,0,3402.660720,53,1,0,f,0,158616.077870,1,3,1


In [57]:
data.loc[data['days_employed'] >= 15000]

,children,days_employed,dob_years,education_id,family_status_id,gender,debt,total_income,income_type_id,purpose_unique_id,family_type_by_children_parents_id
1539,0,15785.678893,59,0,4,f,0,119563.851852,0,0,1
3972,0,15835.725775,64,1,0,f,0,96858.531436,2,3,1
4296,0,17615.563266,61,1,0,f,0,122560.741753,2,0,1
4318,0,15773.061335,61,1,0,f,0,205868.585780,0,3,1
5576,0,15079.216069,55,1,0,f,0,178761.373413,3,0,1
7323,0,16593.472817,60,0,0,f,0,124697.846781,0,2,1
7725,0,15618.063786,64,1,0,f,0,296525.358574,2,2,1
15640,0,15410.040779,65,0,0,f,0,188800.068859,0,0,1
16297,1,18388.949901,61,1,0,f,0,186178.934089,0,0,0
16786,0,16119.687737,64,1,0,f,0,91527.685995,0,0,1


Let's assume that this can happen. A value of 20000 would be critical, but there is no such data, fortunately :)

In [58]:
data.loc[data['days_employed'] == 0]

,children,days_employed,dob_years,education_id,family_status_id,gender,debt,total_income,income_type_id,purpose_unique_id,family_type_by_children_parents_id


Now let's examine the dob_years column.

In [59]:
data.loc[data['dob_years'] >= 100]

,children,days_employed,dob_years,education_id,family_status_id,gender,debt,total_income,income_type_id,purpose_unique_id,family_type_by_children_parents_id


In [60]:
data.loc[data['dob_years'] < 0]

,children,days_employed,dob_years,education_id,family_status_id,gender,debt,total_income,income_type_id,purpose_unique_id,family_type_by_children_parents_id


In [61]:
data.loc[data['dob_years'] == 0]

,children,days_employed,dob_years,education_id,family_status_id,gender,debt,total_income,income_type_id,purpose_unique_id,family_type_by_children_parents_id
99,0,3465.416189,0,1,0,f,0,71291.522491,1,1,1
149,0,2664.273168,0,1,3,f,0,70176.435951,0,0,1
270,3,1872.663186,0,1,0,f,0,102166.458894,0,0,2
578,0,3978.565650,0,1,0,f,0,97620.687042,1,0,1
1040,0,1158.029561,0,0,3,f,0,303994.134987,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...
19770,0,1574.202821,0,1,0,f,0,142594.396847,0,0,1
20397,0,3387.348685,0,1,0,f,0,259193.920299,1,0,1
20512,0,3317.412715,0,1,4,f,0,129788.762899,1,0,1
21110,2,108.967042,0,0,0,m,0,240702.007382,2,0,0


We replace with the mean of age where age is 0.

In [62]:
data.loc[data['dob_years'] == 0, 'dob_years'] = data['dob_years'].mean()
data.loc[data['dob_years'] == 0]

,children,days_employed,dob_years,education_id,family_status_id,gender,debt,total_income,income_type_id,purpose_unique_id,family_type_by_children_parents_id


In [63]:
data.loc[data['total_income'] < 0]

,children,days_employed,dob_years,education_id,family_status_id,gender,debt,total_income,income_type_id,purpose_unique_id,family_type_by_children_parents_id


## Analytics

#### Is there a correlation between having children and paying back the loan on time?

In [64]:
data.groupby('debt')['debt'].count()

debt
0    19713
1     1741
Name: debt, dtype: int64

In [65]:
def is_children(row):
    if row == 0:
        return 'Нет детей'
    return 'Есть дети'

data['is_children'] = data['children'].apply(is_children)
data.head(5)

,children,days_employed,dob_years,education_id,family_status_id,gender,debt,total_income,income_type_id,purpose_unique_id,family_type_by_children_parents_id,is_children
0,1,8437.673028,42.0,0,0,f,0,253875.639453,0,0,0,Есть дети
1,1,4024.803754,36.0,1,0,f,0,112080.014102,0,1,0,Есть дети
2,0,5623.422610,33.0,1,0,m,0,145885.952297,0,0,1,Нет детей
3,3,4124.747207,32.0,1,0,m,0,267628.550329,0,2,2,Есть дети
4,0,3402.660720,53.0,1,0,f,0,158616.077870,1,3,1,Нет детей


In [66]:
def is_debt(row):
    if row == 1:
        return 'Есть задолженность'
    return 'Нет задолженности'

data['is_debt'] = data['debt'].apply(is_debt)
data.head(5)

,children,days_employed,dob_years,education_id,family_status_id,gender,debt,total_income,income_type_id,purpose_unique_id,family_type_by_children_parents_id,is_children,is_debt
0,1,8437.673028,42.0,0,0,f,0,253875.639453,0,0,0,Есть дети,Нет задолженности
1,1,4024.803754,36.0,1,0,f,0,112080.014102,0,1,0,Есть дети,Нет задолженности
2,0,5623.422610,33.0,1,0,m,0,145885.952297,0,0,1,Нет детей,Нет задолженности
3,3,4124.747207,32.0,1,0,m,0,267628.550329,0,2,2,Есть дети,Нет задолженности
4,0,3402.660720,53.0,1,0,f,0,158616.077870,1,3,1,Нет детей,Нет задолженности


In [67]:
data_pivot_children = data.pivot_table(index='is_children', columns='is_debt', values='debt', aggfunc='count')
data_pivot_children

is_debt,Есть задолженность,Нет задолженности
is_children,,
Есть дети,678,6685
Нет детей,1063,13028


Let's find the share in each of the categories (Have children, No children) of the share of loans with arrears in the sum of the total number of loans taken in this category.

In [68]:
data_pivot_children['ratio_expired'] = data_pivot_children['Есть задолженность'] / (data_pivot_children['Нет задолженности'] + data_pivot_children['Есть задолженность'])
data_pivot_children

is_debt,Есть задолженность,Нет задолженности,ratio_expired
is_children,,,
Есть дети,678,6685,0.092082
Нет детей,1063,13028,0.075438


The summary table shows that people who have a child have a higher proportion of debt than those who do not, by 0.016791 (or 1.6791%).

And let's look at the shares for family_type_by_children_parents_id categories.

In [69]:
dict_family_type_by_children_parents

,family_type_by_children_parents_id,family_type_by_children_parents
0,0,полная малодетная семья
1,1,Не семья
2,2,полная многодетная семья
3,3,неполная малодетная семья
4,4,неполная многодетная семья


In [70]:
data_pivot_children_2 = data.pivot_table(index='family_type_by_children_parents_id', columns='is_debt', values='debt', aggfunc='count')
data_pivot_children_2['ratio_expired'] = data_pivot_children_2['Есть задолженность'] / (data_pivot_children_2['Есть задолженность'] + data_pivot_children_2['Нет задолженности'])
data_pivot_children_2.sort_values(by='ratio_expired', ascending=False)

is_debt,Есть задолженность,Нет задолженности,ratio_expired
family_type_by_children_parents_id,,,
0,608,5871,0.093842
2,30,331,0.083102
3,39,465,0.077381
1,1063,13028,0.075438
4,1,18,0.052632


This summary table shows that full families with few children have the highest share of arrears (9.3654%). The second place is occupied by complete large families (8.3102%). In the third place are incomplete small families (7.7381%). In fourth is non-family (7.5129%). And in fifth - incomplete large families (5.2632%).

#### Conclusion

We conclude that those who have children are more prone to have loan arrears. Complete few children and large families are more prone to have loan arrears. After incomplete small families and non-families have the same proportion of arrears. And incomplete large families are least likely to be in debt. Interesting, interesting :)

#### Is there a correlation between marital status and loan repayment on time?

In [71]:
data_pivot_family_status = data.pivot_table(index='family_status_id', columns='is_debt', values='debt', aggfunc='count')
data_pivot_family_status['ratio_expired'] = data_pivot_family_status['Есть задолженность'] / (data_pivot_family_status['Есть задолженность'] + data_pivot_family_status['Нет задолженности'])
data_pivot_family_status.sort_values(by='ratio_expired', ascending=False)

is_debt,Есть задолженность,Нет задолженности,ratio_expired
family_status_id,,,
4,274,2536,0.097509
0,1319,15171,0.079988
3,85,1110,0.071130
2,63,896,0.065693


In [72]:
dict_family_status

,family_status,family_status_id
0,женат / замужем,0
1,вдовец / вдова,2
2,в разводе,3
3,не женат / не замужем,4


#### Conclusion

From the summary table we can conclude that unmarried/unmarried people (9.7405%) are more prone to indebtedness. People who are either married or in civil marriage have a share of debt of 7.9664%. Almost the same percentage is for people who are divorced (7.113%). People who are widows/widowers are least likely to be in debt - 6.5625%.

### Is there a correlation between income level and loan repayment on time?

To find out, we need to procategorize people by income level.

In [73]:
def level_income(column_value):
    if column_value <= 100000:
        return 'низкий доход'
    if 100000 < column_value <= 300000:
        return 'средний доход'
    return 'высокий доход'

data['level_income'] = data['total_income'].apply(level_income)
data.head(10)

,children,days_employed,dob_years,education_id,family_status_id,gender,debt,total_income,income_type_id,purpose_unique_id,family_type_by_children_parents_id,is_children,is_debt,level_income
0,1,8437.673028,42.0,0,0,f,0,253875.639453,0,0,0,Есть дети,Нет задолженности,средний доход
1,1,4024.803754,36.0,1,0,f,0,112080.014102,0,1,0,Есть дети,Нет задолженности,средний доход
2,0,5623.422610,33.0,1,0,m,0,145885.952297,0,0,1,Нет детей,Нет задолженности,средний доход
3,3,4124.747207,32.0,1,0,m,0,267628.550329,0,2,2,Есть дети,Нет задолженности,средний доход
4,0,3402.660720,53.0,1,0,f,0,158616.077870,1,3,1,Нет детей,Нет задолженности,средний доход
5,0,926.185831,27.0,0,0,m,0,255763.565419,2,0,1,Нет детей,Нет задолженности,средний доход
6,0,2879.202052,43.0,0,0,f,0,240525.971920,2,0,1,Нет детей,Нет задолженности,средний доход
7,0,152.779569,50.0,1,0,m,0,135823.934197,0,2,1,Нет детей,Нет задолженности,средний доход
8,2,6929.865299,35.0,0,0,f,0,95856.832424,0,3,0,Есть дети,Нет задолженности,низкий доход
9,0,2188.756445,41.0,1,0,m,0,144425.938277,0,0,1,Нет детей,Нет задолженности,средний доход


In [74]:
data['level_income'].value_counts()

средний доход    15508
низкий доход      4463
высокий доход     1483
Name: level_income, dtype: int64

In [75]:
data_pivot_level_income = data.pivot_table(index='level_income', columns='is_debt', values='debt', aggfunc='count')
data_pivot_level_income['ratio_expired'] = data_pivot_level_income['Есть задолженность'] / (data_pivot_level_income['Есть задолженность'] + data_pivot_level_income['Нет задолженности'])
data_pivot_level_income.sort_values(by='ratio_expired', ascending=False)

is_debt,Есть задолженность,Нет задолженности,ratio_expired
level_income,,,
средний доход,1281,14227,0.082603
низкий доход,354,4109,0.079319
высокий доход,106,1377,0.071477


#### Conclusion

We see that people with high income have the lowest share of debt - 7.1477%. The middle class is most prone to debt at 8.2226%. The percentage of low-income people who are in credit debt is 7.9319%.

### How do different purposes of a loan affect its repayment on time?

In [76]:
data_pivot_level_income = data.pivot_table(index='purpose_unique_id', columns='is_debt', values='debt', aggfunc='count')
data_pivot_level_income['ratio_expired'] = data_pivot_level_income['Есть задолженность'] / (data_pivot_level_income['Есть задолженность'] + data_pivot_level_income['Нет задолженности'])
data_pivot_level_income.sort_values(by='ratio_expired', ascending=False)

is_debt,Есть задолженность,Нет задолженности,ratio_expired
purpose_unique_id,,,
1,403,3903,0.093590
2,370,3643,0.092200
3,186,2138,0.080034
0,782,10029,0.072334


In [77]:
dict_purpose_unique

,purpose_unique_id,purpose_unique
0,0,недвижимость
1,1,автомобиль
2,2,образование
3,3,свадьба


#### Conclusion

People whose purpose is an automobile have a higher percentage of debt - 9.3395%. People with the purpose of education have almost the same percentage at 9.1994%. People with the purpose of wedding and real estate have a lower propensity for debt, with 7.9216% in one case and 7.214% in the other.

## General output

Putting the data in order, their analysis showed that the people most prone to credit debt are those with middle income, with children, single/unmarried, whose goal is a car or education.

Questions and hypotheses to test:
- Having children increases the likelihood of getting into credit debt - 9.2%;
- complete families are most prone to credit debt - 8-9%;
- unmarried/ single people are most likely to incur credit debt - 9.7%;
- the middle class is most prone, like the low-income class, to credit debt - 8%;
- people whose goal is to pay for education and buy a car are most prone to credit debt - 9%.

The bank should keep the findings in mind when deciding whether to extend credit to people in these categories. As an option, reduce the number of loans to people in these categories.